# Проект по SQL

Наша компания купила сервис для чтения книг по подписке. Наша задача - проанализировать базу данных, где содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. На основе этих данных впоследствии можно будет сформулировать ценностное предложение для нового продукта.

## Содержание
* [Выгрузка данных и предпросмотр таблиц](#выгрузка)
* [Выполнение заданий](#задания)
    * [Количество книг, вышедших после 1 января 2000 года](#книги_2000)
    * [Средняя оценка и количество обзоров у каждой книги](#оценка_обзоры)
    * [Издательство, которые выпустило наибольшее количество книг толце 50 страниц](#издательство)
    * [Автор с самой высокой средней оценкой книг](#автор)
    * [Среднее количество обзоров пользователей, которые поставили больше 50 оценок](#обзоры)

### Выгрузка данных и предпросмотр таблиц <a class="anchor" id="выгрузка"></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# создадим функцию для просмотра первых пяти строк и информации о таблице
def sql_info(sql):
    display(pd.io.sql.read_sql('''SELECT * FROM {} LIMIT 5'''.format(sql), con = engine))
    display()
    display(pd.io.sql.read_sql('''SELECT * FROM {} '''.format(sql), con = engine).info())

In [3]:
# сначала таблица books
sql_info('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

1000 строк, 6 столбцов. Есть id книги и автора, количество страниц, название, дата публикации и id издателя.

In [4]:
# теперь информация об авторах 
sql_info('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

636 строк, 2 столбца с id автора и его/ее именем. Может встречаться несколько авторов под одним id. 

In [5]:
# издательства 
sql_info('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

Здесь 340 строк, 2 столбца с названием издательства и его id. 

In [6]:
# пользовательские обзоры книг
sql_info('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

2793 строки и 4 столбца с информацией о id обзора и книги, имени пользователя и сам текст.

In [7]:
# пользовательские оценки
sql_info('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

4 столбца и 6456 строк. Есть данные с id рейтинга и книги, именем пользователя и сам рейтинг. 

Итого у нас 5 связанных между собой таблиц, в которых содержится информация о книгах, авторах, издательствах, пользовательских оценках и отзывах. Так как предобработка не требовалась, мы не проверяли ни на дубликаты, ни на пропуски. Но, судя по всему, пропусков нет. 

### Выполнение заданий <a class="anchor" id="задания"></a>

#### Количество книг, вышедших после 1 января 2000 года <a class="anchor" id="книги_2000"></a>

In [8]:
book_count = '''
    SELECT COUNT(book_id) AS book_count
    FROM books
    WHERE publication_date::date > '2000-01-01'
    '''

In [9]:
pd.io.sql.read_sql(book_count, con = engine)

,book_count
0,819


819 книг было опубликовано после 1 января 2020. Всего в таблице 1000 книг, так что под запрос попало большинство. 

#### Средняя оценка и количество обзоров у каждой книги <a class="anchor" id="оценка_обзоры"></a>

In [10]:
rating_and_review_count = '''
    SELECT b.book_id as id,
           b.title as title,
           ROUND(AVG(rat.rating),2) as avg_rating,
           COUNT(DISTINCT rev.review_id) as review_count
    FROM books as b
    LEFT JOIN ratings as rat ON b.book_id = rat.book_id
    LEFT JOIN reviews as rev ON b.book_id = rev.book_id
    GROUP BY id
    ORDER BY avg_rating DESC
    '''

In [11]:
pd.io.sql.read_sql(rating_and_review_count, con = engine)

,id,title,avg_rating,review_count
0,86,Arrows of the Queen (Heralds of Valdemar #1),5.00,2
1,901,The Walking Dead Book One (The Walking Dead #...,5.00,2
2,390,Light in August,5.00,2
3,972,Wherever You Go There You Are: Mindfulness Me...,5.00,2
4,136,Captivating: Unveiling the Mystery of a Woman'...,5.00,2
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3
996,316,His Excellency: George Washington,2.00,2
997,202,Drowning Ruth,2.00,3
998,371,Junky,2.00,2


У нескольких книг средний рейтинг 5, самый низкий рейтинг у Harvesting the Heart - 1.5. 

#### Издательство, которое выпустило наибольшее количество книг толще 50 страниц <a class="anchor" id="издательство"></a>

In [12]:
publisher = '''
    SELECT p.publisher as publisher,
        COUNT(DISTINCT b.book_id) as book_count
    FROM books AS b
    JOIN publishers as p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY COUNT(DISTINCT b.book_id) DESC
    LIMIT 5
    '''

In [13]:
pd.io.sql.read_sql(publisher, con = engine)

,publisher,book_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


Лидирует одно из самых популярных издательств - Penguin Books с 42 книгами. 

#### Автор с самой высокой средней оценкой книг <a class="anchor" id="автор"></a>

У книг автора должно быть более 50 оценок

In [14]:
best_author = '''
    SELECT t.author as author,
        ROUND(AVG(t.rating), 2) as rating
    FROM 
        (SELECT a.author as author,
                b.book_id as book,
                AVG(r.rating) as rating,
                COUNT(r.rating) as rating_count
        FROM books as b
        JOIN authors as a on b.author_id = a.author_id
        JOIN ratings as r on b.book_id = r.book_id
        GROUP BY author, book
        HAVING COUNT(r.rating) > 50) AS t
    GROUP BY author
    ORDER BY rating DESC
    LIMIT 5
    '''

In [15]:
pd.io.sql.read_sql(best_author, con = engine)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08


Автор с самым высоким рейтингом - Дж.К. Роулинг. Мэри Гранпрэ - иллюстратор книг о Гарри Поттере.

#### Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок <a class="anchor" id="обзоры"></a>

In [16]:
active_users = '''
    SELECT ROUND(AVG(review_cnt)) as avg_review
    FROM 
        (SELECT COUNT(review_id) as review_cnt
        FROM reviews 
        WHERE username IN
            (SELECT username
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating_id) > 50)
        GROUP BY username) as u
    '''

In [17]:
pd.io.sql.read_sql(active_users, con = engine)

,avg_review
0,24.0


В среднем активные пользователи (которые поставили больше 50 оценок) пишут 24 отзыва. 

**Выводы**

1. 819 книг вышли после 1 января 2020 года. Это большая часть книг из той БД, что у нас есть.
2. Самая высокая средняя оценка по пятибалльной шкале у книг - 5, самая низкая - 1.5. При этом отзывов на каждую такую книгу немного - всего 2. 
3. Больше всего выпущенных книг у популярного британского издательства Penguin Books - 42 книги объемом более 50 страниц. Кроме того, в топ-5 издательств входит серия Penguin Books - Penguin Classics с 24 выпущенными книгами.
4. Автор с самым высоким рейтингом книг - Дж.К. Роулинг (и иллюстратор книг о Гарри Поттере Мэри Гранпрэ). Средний рейтинг ее книг достигает 4.28. 
5. Те пользователи, которые оставили более 50 оценок, в среднем пишут 24 отзыва, что достаточно много.